In [2]:
import torch

from maskrcnn_benchmark.structures.bounding_box import BoxList

In [26]:
n = 6
box = torch.randint(2, 100, (n, 4))
scores = torch.rand(n)
ids_valid = torch.arange(0, n // 2)
ids_invalid = torch.full((n // 2,), -1)
ids = torch.cat((ids_valid, ids_invalid))
ids

boxlist = BoxList(box, (800, 600))
boxlist.add_field('scores', scores)
boxlist.add_field('ids', ids)
boxlist

BoxList(num_boxes=6, image_width=800, image_height=600, mode=xyxy)

In [32]:
def boxlist_select_tensor_subset(boxlist: BoxList, selector) -> BoxList:
    box = boxlist.bbox[selector]
    scores = boxlist.get_field('scores')[selector]
    ids = boxlist.get_field('ids')[selector]

    boxlist = BoxList(box, boxlist.size, boxlist.mode)
    boxlist.add_field('scores', scores)
    boxlist.add_field('ids', ids)

    return boxlist

In [34]:
selector = boxlist.get_field('ids') < 0
boxlist_subset = boxlist_select_tensor_subset(boxlist, selector)
boxlist_subset

BoxList(num_boxes=3, image_width=800, image_height=600, mode=xyxy)

In [39]:
import numpy as np

from scipy.optimize import linear_sum_assignment

cost_matrix = np.random.randint(0, 5, (5, 3))
cost_matrix

array([[4, 4, 0],
       [3, 0, 1],
       [4, 1, 0],
       [0, 2, 4],
       [1, 2, 0]])

In [41]:
linear_sum_assignment(cost_matrix)

(array([0, 1, 3], dtype=int64), array([2, 1, 0], dtype=int64))

In [9]:
def square_and_cube(x):
    return x ** 2, x ** 3

def squares_sum_access_index():
    sum = 0
    for x in range(10_000):
        cube = square_and_cube(x)[1]
        sum += cube
    return sum

def squares_sum_access_tuple():
    sum = 0
    for x in range(10_000):
        _, cube = square_and_cube(x)
        sum += cube
    return sum

In [10]:
%timeit squares_sum_access_index

28.1 ns ± 2.71 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [34]:
import torch

In [44]:
m = 4
n = 5
a = torch.randint(0, 3, (m, 8)).float()
b = torch.randint(0, 3, (n, 8)).float()

a, b

(tensor([[0., 0., 1., 2., 0., 1., 0., 2.],
         [2., 2., 0., 0., 0., 2., 0., 1.],
         [0., 0., 0., 1., 0., 0., 2., 0.],
         [1., 0., 0., 2., 1., 1., 1., 0.]]),
 tensor([[2., 0., 0., 2., 0., 2., 0., 1.],
         [0., 0., 0., 1., 0., 2., 2., 2.],
         [2., 0., 0., 1., 0., 0., 2., 2.],
         [1., 1., 2., 0., 1., 2., 2., 1.],
         [2., 2., 2., 0., 0., 0., 2., 1.]]))

In [52]:
from torch.nn.functional import cosine_similarity

def cos_sim_matrix(a, b, eps=1e-8):
    a_n = a.norm(dim=1)[..., None]
    b_n = b.norm(dim=1)[..., None]
    a_norm = a / torch.clamp(a_n, min=eps)
    b_norm = b / torch.clamp(b_n, min=eps)
    sim_matrix = torch.mm(a_norm, b_norm.transpose(0, 1))
    return sim_matrix

a / a.norm(dim=1)[..., None]

tensor([[0.0000, 0.0000, 0.3162, 0.6325, 0.0000, 0.3162, 0.0000, 0.6325],
        [0.5547, 0.5547, 0.0000, 0.0000, 0.0000, 0.5547, 0.0000, 0.2774],
        [0.0000, 0.0000, 0.0000, 0.4472, 0.0000, 0.0000, 0.8944, 0.0000],
        [0.3536, 0.0000, 0.0000, 0.7071, 0.3536, 0.3536, 0.3536, 0.0000]])